### Àlex Correa Orri 1564967

#### Dataset: https://www.kaggle.com/shaurov/website-classification-using-url

In [180]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details


### MÒDULS I LLIBRERIES

In [181]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats
import os
import seaborn as sns
from scipy.stats import shapiro, spearmanr
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipympl
%matplotlib widget

### DIRECTORI DE TREBALL

In [182]:
os.getcwd()
path = 'C:/Users/alexc/OneDrive/Escritorio/UNI/3er/Aprenentatge Computacional/PRAC_KAGGLE'
os.chdir(path)
os.getcwd()
# Visualitzarem només 3 decimals per mostra
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

'C:\\Users\\alexc\\OneDrive\\Escritorio\\UNI\\3er\\Aprenentatge Computacional\\PRAC_KAGGLE'

### LECTURA DE DADES

In [183]:
# Carreguem dataset d'exemple
dataset = pd.read_csv('URL_Classification.csv', names=['n', 'url', 'class'], delimiter=',')

In [184]:
#dataset

#### Carreguem un dataset amb més de 466k paraules en anglès: https://github.com/dwyl/english-words

In [185]:
def load_words():
    with open('words.txt') as word_file:
        valid_words = (word_file.read().split())

    return valid_words
l_words = load_words()

### MODIFICACIÓ DEL DATASET

In [186]:
dataset = dataset.drop(['n'], axis=1)

In [187]:
#dataset

In [188]:
#dataset['class'].unique()

#### Per a fer proves reduim el sample a 100000 dades

In [189]:
s=10000
dataset1 = dataset.sample(n=s, random_state=10)

In [190]:
#dataset1['class'].unique()

In [ ]:
l = [0]*s
for w in l_words:
    dataset1[w] = l